# Experiment 001: Baseline with Arrhenius Kinetics + TTA

Implementing the top solution approach:
1. Arrhenius kinetics features (1/T, ln(t), interaction)
2. Spange descriptors for solvent featurization
3. MLP with BatchNorm, Dropout, HuberLoss
4. Model bagging (7 seeds)
5. Test Time Augmentation for chemical symmetry

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

GPU available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 85.0 GB
Using device: cuda


In [2]:
# Load data - adapted for local paths
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def load_features(name='spange_descriptors'):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

# Load spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f"Spange descriptors shape: {SPANGE_DF.shape}")
print(f"Solvents: {list(SPANGE_DF.index[:5])}...")

Spange descriptors shape: (26, 13)
Solvents: ['Cyclohexane', 'Ethyl Acetate', 'Acetic Acid', '2-Methyltetrahydrofuran [2-MeTHF]', '1,1,1,3,3,3-Hexafluoropropan-2-ol']...


In [3]:
# Kinetic Featurizer with Arrhenius features
class KineticMixingFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        # Features: rt, temp, inv_temp, log_time, interaction + solvent features
        self.feats_dim = self.featurizer.shape[1] + 5

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # --- KINETIC ENGINEERING (Arrhenius) ---
        temp_c = X_vals[:, 1:2]  # Temperature in Celsius
        time_m = X_vals[:, 0:1]  # Residence time in minutes
        
        temp_k = temp_c + 273.15  # Convert to Kelvin
        inv_temp = 1000.0 / temp_k  # Inverse temperature (scaled)
        log_time = np.log(time_m + 1e-6)  # Log time
        interaction = inv_temp * log_time  # Arrhenius interaction term
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # --- SOLVENT FEATURES ---
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # SYMMETRY FLIP: Swap A and B
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)

In [4]:
# MLP Architecture
class MLPInternal(nn.Module):
    def __init__(self, input_dim):
        super(MLPInternal, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3),
            nn.Sigmoid()  # Bounded outputs [0, 1]
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# Symmetric Bagged Model
class SymmetricBaggedModel(nn.Module):
    def __init__(self, data='single', n_models=7):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticMixingFeaturizer(mixed=(data=='full'))
        self.n_models = n_models
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, epochs=300, verbose=False):
        # 1. Standard Data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # 2. Augmented Data (Train on both symmetries)
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)  # Different seed for each model
            np.random.seed(42 + i)
            
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
            criterion = nn.HuberLoss()  # Robust to outliers
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        if len(self.models) == 0:
            raise ValueError("Model not trained yet")
            
        # --- TEST TIME AUGMENTATION (TTA) ---
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
            
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

In [ ]:
# Test on a single fold first to verify everything works
print("Testing on single fold...")
X, Y = load_data("single_solvent")
print(f"Single solvent data: {X.shape[0]} samples")

# Get first fold
split_gen = generate_leave_one_out_splits(X, Y)
(train_X, train_Y), (test_X, test_Y) = next(split_gen)

print(f"Train: {train_X.shape[0]}, Test: {test_X.shape[0]}")

# Train model with fewer epochs for testing
model = SymmetricBaggedModel(data='single', n_models=3)
model.train_model(train_X, train_Y, epochs=50)

# Predict
preds = model.predict(test_X)
print(f"Predictions shape: {preds.shape}")
print(f"Sample predictions: {preds[:3]}")

# Calculate MSE for this fold
mse = ((preds.numpy() - test_Y.values) ** 2).mean()
print(f"Fold MSE: {mse:.6f}")

In [ ]:
# Full CV evaluation - Task 0: Single Solvent
print("\n" + "="*60)
print("TASK 0: Single Solvent (Leave-One-Solvent-Out CV)")
print("="*60)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions_task0 = []
fold_mses_task0 = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='single', n_models=7)
    model.train_model(train_X, train_Y, epochs=300)
    
    predictions = model.predict(test_X).numpy()
    
    # Calculate fold MSE
    fold_mse = ((predictions - test_Y.values) ** 2).mean()
    fold_mses_task0.append(fold_mse)
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions_task0.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

print(f"\nTask 0 Mean MSE: {np.mean(fold_mses_task0):.6f} +/- {np.std(fold_mses_task0):.6f}")

In [ ]:
# Full CV evaluation - Task 1: Full Data with Mixtures
print("\n" + "="*60)
print("TASK 1: Full Data (Leave-One-Ramp-Out CV)")
print("="*60)

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_task1 = []
fold_mses_task1 = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='full', n_models=7)
    model.train_model(train_X, train_Y, epochs=300)
    
    predictions = model.predict(test_X).numpy()
    
    # Calculate fold MSE
    fold_mse = ((predictions - test_Y.values) ** 2).mean()
    fold_mses_task1.append(fold_mse)
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions_task1.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

print(f"\nTask 1 Mean MSE: {np.mean(fold_mses_task1):.6f} +/- {np.std(fold_mses_task1):.6f}")

In [ ]:
# Calculate overall score
overall_mse = (np.mean(fold_mses_task0) + np.mean(fold_mses_task1)) / 2
print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Task 0 (Single Solvent) MSE: {np.mean(fold_mses_task0):.6f}")
print(f"Task 1 (Full Data) MSE: {np.mean(fold_mses_task1):.6f}")
print(f"Overall Score (avg): {overall_mse:.6f}")
print(f"Target to beat: 0.017270")

In [ ]:
# Create submission file
import os
os.makedirs('/home/submission', exist_ok=True)

submission_single = pd.DataFrame(all_predictions_task0)
submission_full = pd.DataFrame(all_predictions_task1)
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

print(f"Submission saved to /home/submission/submission.csv")
print(f"Submission shape: {submission.shape}")
print(submission.head())